In [1]:
import pandas as pd
import unicodedata

In [2]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [3]:
populacao = pd.read_csv("../data/populacao/populacao.csv")

In [4]:
populacao.Cidade = populacao.Cidade.apply(lambda x: x[:-5])

In [5]:
regioes = pd.read_csv("../data/geoloc/regioes-saude.csv")

In [6]:
regioes.municipio = regioes.municipio.apply(lambda x: remove_accents(x).lower().strip())

In [7]:
regioes.set_index("municipio",inplace=True)

In [8]:
def get_ibge_code(cityname): 
    try:
        return regioes.loc[remove_accents(cityname).lower().replace("-", " "),"CODIBGE"]
    except KeyError:
        print(f"Erro com {cityname}")
        return None

In [9]:
populacao["COD_IBGE"] = populacao.Cidade.apply(lambda x: get_ibge_code(x)) 

In [10]:
populacao.set_index("COD_IBGE",inplace=True)

In [11]:
populacao.to_csv("../parsed-data/populacao.csv")

Teste

In [29]:
doencas = pd.read_csv("../parsed-data/doencas.csv")

In [33]:
def get_incidencia(doenca, ano):
    global doencas
    serie =  doencas.query(f"ano == {ano}").set_index("Ibge")[doenca]
    a = serie.div(populacao[f"{ano}"])
    return pd.DataFrame({"IBGE": a.index.values, "incidência": a.values})

In [34]:
get_incidencia("Hipertensão", 2014)

,IBGE,incidência
0,420005,NaN
1,420010,NaN
2,420020,NaN
3,420030,NaN
4,420040,NaN
...,...,...
290,421950,0.000378
291,421960,NaN
292,421970,0.000366
293,421985,NaN


In [35]:
populi =  pd.read_csv("../parsed-data/populacao.csv")

In [38]:
doencas

,Unnamed: 0,Ibge,Hipertensão,Diabetes,Obesidade,Tabagismo,ano
0,0,420915,87,39,5,1,2014
1,1,420370,0,1,17,1,2014
2,2,421550,4,3,2,10,2014
3,3,420630,15,3,3,0,2014
4,4,420530,49,18,69,0,2014
...,...,...,...,...,...,...,...
1995,290,420757,1622,385,110,4,2021
1996,291,420980,436,129,19,19,2021
1997,292,421400,1896,632,334,128,2021
1998,293,420253,930,159,107,83,2021
